In [1]:
import os, numpy as np, logging
from pyapphot.apphot import starPSF
from pyapphot.utils import store_filenames
os.chdir('test_data/jcbt/')

Ginga not installed, use other viewer, or no viewer


# Calculate PSF properties at given co-ordinates
PSF properties which are required for checking the quality of the images and also for photometry are:\
FWHM, peak,ellipticity, pa ,background-sky noise. You can calculate all the properties using the starPSF class and save the results to a pickle file which can be opened later.

In [2]:
psf = starPSF(image='albf*.fits', coordfile='new_bf_07jan2019_wasp50_R_030.coo', calculate=True, saveto='imstat.pkl') 

xc=948.6236	yc=2932.7997
xc=1089.9807	yc=688.0503
xc=846.4712	yc=204.7184
xc=1095.5463	yc=2272.3428
xc=948.6176	yc=2932.7317
xc=1089.8859	yc=688.2036
xc=846.4329	yc=204.8784
xc=1095.5365	yc=2272.1975
xc=948.6481	yc=2932.6049
xc=1089.9644	yc=688.0907
xc=846.4599	yc=204.8185
xc=1095.5373	yc=2272.1431
xc=948.7265	yc=2932.5301
xc=1089.9702	yc=688.2495
xc=846.4159	yc=204.9176
xc=1095.6041	yc=2272.1295
xc=948.6383	yc=2932.6270
xc=1089.9783	yc=688.2095
xc=846.5224	yc=204.9318
xc=1095.6257	yc=2272.2149
xc=948.6534	yc=2932.6489
xc=1089.9837	yc=687.9903
xc=846.3963	yc=204.6261
xc=1095.5980	yc=2272.0137


# Load PSF stat and analyze image quality

In [2]:
psf =  starPSF(image='al@imlist', coordfile='new*.coo', loadfrom='imstat.pkl')
psfstats = psf.get_stat('all', statkey=['ellip','peak','fwhm'], return_dict=True) # check some properties
# or just
# starPSF.get_stat(None,statkey=['ellip','peak','fwhm']) # check some properties
print(psfstats)

{'ellip': array([[0.02, 0.04, 0.02, 0.02],
       [0.02, 0.02, 0.01, 0.01],
       [0.04, 0.01, 0.  , 0.01],
       [0.08, 0.03, 0.04, 0.05],
       [0.05, 0.04, 0.03, 0.04],
       [0.04, 0.02, 0.05, 0.05]]), 'peak': array([[ 4674.16086318,  3197.17871246,  9927.81312818, 20023.64846985],
       [ 5120.46067834,  3476.07143611, 10790.61522491, 21647.95801661],
       [ 3865.43662279,  2610.37614436,  8074.01089724, 15905.78418324],
       [ 2810.58260038,  2006.17599869,  6299.5766085 , 11716.59031954],
       [ 2749.35874714,  1881.76802317,  5981.40939224, 11819.89013681],
       [ 2270.23808048,  1603.466401  ,  4981.87262403,  9859.03201226]]), 'fwhm': array([[6.436, 6.224, 6.264, 6.164],
       [5.96 , 5.988, 5.968, 5.78 ],
       [6.8  , 6.66 , 6.816, 6.672],
       [7.636, 7.272, 7.196, 7.392],
       [7.544, 7.42 , 7.372, 7.236],
       [9.139, 7.796, 7.884, 7.744]]), 'image': ['albf07jan2019_wasp50_R_030.fits', 'albf07jan2019_wasp50_R_031.fits', 'albf07jan2019_wasp50_R_032.fi

You can impose criteria and select the objects and frames accordingly. You can set a trade-off between removing bad objects and bad frames by the argument 'Nfiledelmax'. This tells you how many frames you are ready to remove from your calculation at most. Accordingly the objects not matching the criteria will be removed.This analysis will give you a list of frames that satisfy the given criteria for each object and also tells you how many frames and how many objects you need to remove from your calculation.

In [3]:
criteria = dict(ellip=(0,0.13),peak=(2000,50000))
imselected, remove = psf.select_imageNobj(criteria, objsynced=True, Nfiledelmax=2)
print("\nFollowing files are selected per object\n")
for i,im in enumerate(imselected):
    print(f"Object {i+1}: {', '.join(im)}")
remove

object 0: conditions met by all the images.
object 2: conditions not met by 2 image(s).
object 2: conditions met by all the images.
object 3: conditions met by all the images.

Following files are selected per object

Object 1: albf07jan2019_wasp50_R_030.fits, albf07jan2019_wasp50_R_031.fits, albf07jan2019_wasp50_R_032.fits, albf07jan2019_wasp50_R_033.fits, albf07jan2019_wasp50_R_034.fits, albf07jan2019_wasp50_R_035.fits
Object 2: albf07jan2019_wasp50_R_030.fits, albf07jan2019_wasp50_R_031.fits, albf07jan2019_wasp50_R_032.fits, albf07jan2019_wasp50_R_033.fits
Object 3: albf07jan2019_wasp50_R_030.fits, albf07jan2019_wasp50_R_031.fits, albf07jan2019_wasp50_R_032.fits, albf07jan2019_wasp50_R_033.fits, albf07jan2019_wasp50_R_034.fits, albf07jan2019_wasp50_R_035.fits
Object 4: albf07jan2019_wasp50_R_030.fits, albf07jan2019_wasp50_R_031.fits, albf07jan2019_wasp50_R_032.fits, albf07jan2019_wasp50_R_033.fits, albf07jan2019_wasp50_R_034.fits, albf07jan2019_wasp50_R_035.fits


{'delete_images': ['albf07jan2019_wasp50_R_035.fits',
  'albf07jan2019_wasp50_R_034.fits'],
 'remove_obj': []}

In [10]:
# psf.adjust_coordfile(remove['remove_obj'])

In [4]:
store_filenames('albf*.fits', remove['delete_images'], 'photimlist')